## Using News API we can get more information about the market
The Natural Language Toolkit was used to analyze the sentiment of articles collected for a given topic. These sentiments were grouped by date in order to see the volumne and opinion about the keyword on a given day. We chose the keyword "stocks" here but played around with other keywords.

#### Shout out to my boy Jordan of newsapi.org for giving me access back. They changed their access policy on monday from giving me years of data to only giving me a 100 articles.

In [1]:
NEWS_API_KEY = "c07df8265f264b20a0c2646a1740193e"
import requests
import json

import pandas as pd
from pandas.io.json import json_normalize
pd.options.display.max_rows = 10
import re
import nltk.sentiment.util

In [2]:
def get_all_articles(topic, date):
    page_number = 1
    all_articles = pd.DataFrame({})
    
    # TODO fuck they changed their API
    while page_number <= 20:
        url = ('https://newsapi.org/v2/everything?'
           'q=%s&'
           'from=%s&'
           'page=%d&'
           'language=en&'  # language is english
           'pageSize=100&'  # maximum page size
           'sortBy=popularity&'
           'apiKey=%s'
        ) % (topic, date, page_number, NEWS_API_KEY)
        response = requests.get(url)
        news = json.loads(response.text)
        print(news)
        if not news["articles"]:
            break
        all_articles = pd.concat([all_articles, json_normalize(news["articles"])], ignore_index=True)
        page_number += 1
        print(len(all_articles))
    return all_articles

def clean_articles(df):
    df["Date"] = df.publishedAt.str.split("T").str[0]
    df.dropna(subset=['description'], inplace=True)
    return df

In [3]:
import sys
from io import StringIO

def get_sentiment(articles):
    s = StringIO()
    sys.stdout = s
    articles["description"].apply(nltk.sentiment.util.demo_liu_hu_lexicon)
    # WHY ON EARTH WOULD YOU WRITE A FUNCTION THAT PRINTS INSTEAD OF RETURNING?
    sys.stdout = sys.__stdout__
    return s.getvalue().splitlines()

def get_sentiment_by_date(articles):
    return articles.groupby(["Date","sentiment"]).size().unstack(fill_value=0)

In [ ]:
articles = get_all_articles("stocks", "2017-05-14")
articles

In [ ]:
articles

In [46]:
articles = clean_articles(articles)

In [47]:
articles["sentiment"] = get_sentiment(articles)

In [41]:
articles.head()

,author,description,publishedAt,source.id,source.name,title,url,urlToImage,Date,sentiment
0,"Alicia Adamczyk on Two Cents, shared by Alicia...",So you’re a freelancer and you want to know wh...,2018-05-25T16:30:00Z,None,Lifehacker.com,How Freelancers Can Save for Retirement Beyond...,https://twocents.lifehacker.com/how-freelancer...,https://i.kinja-img.com/gawker-media/image/upl...,2018-05-25,Positive
1,Mariella Moon,Tesla has asked a federal court in San Francis...,2018-05-27T01:29:00Z,engadget,Engadget,Tesla denies exaggerating Model 3 production p...,https://www.engadget.com/2018/05/26/tesla-mode...,https://o.aolcdn.com/images/dims?thumbnail=120...,2018-05-27,Negative
2,Brian Heater,Baidu’s stocks took a hit earlier today after ...,2018-05-18T18:07:52Z,techcrunch,TechCrunch,Baidu’s top AI exec is stepping down,http://techcrunch.com/2018/05/18/baidus-top-ai...,https://techcrunch.com/wp-content/uploads/2018...,2018-05-18,Neutral
3,Matt Burns,Today at Apple’s World Wide Developer Conferen...,2018-06-04T19:26:02Z,techcrunch,TechCrunch,Apple to bring iOS apps to macOS,http://techcrunch.com/2018/06/04/apple-is-brin...,https://techcrunch.com/wp-content/uploads/2018...,2018-06-04,Positive
4,Sarah Perez,Target is lowering the price of its next-day d...,2018-05-15T14:01:00Z,techcrunch,TechCrunch,"Target’s next-day delivery service, Target Res...",http://techcrunch.com/2018/05/15/targets-next-...,https://techcrunch.com/wp-content/uploads/2014...,2018-05-15,Positive


In [32]:
sentiment_by_date = get_sentiment_by_date(articles)
sentiment_by_date.columns
sentiment_by_date

sentiment,Negative,Neutral,Positive
Date,,,
2018-05-14,0,0,1
2018-05-15,4,0,1
2018-05-16,2,1,1
2018-05-17,0,1,0
2018-05-18,0,1,0
...,...,...,...
2018-06-08,2,0,1
2018-06-10,0,1,0
2018-06-11,0,0,1


In [20]:
import pickle
pickle.dump(sentiment_by_date, open("sentiment_by_date.pkl", "wb"))

In [61]:
date1 = "2017-06-15"
url = ('https://newsapi.org/v2/everything?'
           'q=%s&'
           'from=%s&'
           'page=%d&'
           'language=en&'  # language is english
           'pageSize=10&'  # maximum page size
           'sortBy=popularity&'
           'apiKey=%s'
           
        ) % ("stocks", date1, 1, NEWS_API_KEY)
response = requests.get(url)
# news = json.loads(response.text)
response.text

'{"status":"ok","totalResults":222263,"articles":[{"source":{"id":null,"name":"Lifehacker.com"},"author":"Patrick Allan","title":"What Is a \\"Bump Stock\\" And Why Is It so Dangerous?","description":"The shooter who recently terrorized the Las Vegas strip, leaving 59 dead and hundreds injured, used gun modifications called “bump stocks” to make his rifles even deadlier. But what are they exactly? Read more...","url":"https://lifehacker.com/what-is-a-bump-stock-and-why-is-it-so-dangerous-1819180018","urlToImage":"https://i.kinja-img.com/gawker-media/image/upload/s--0m7CTp18--/c_fill,fl_progressive,g_center,h_450,q_80,w_800/dj0ewuz1s8nmscib6yhq.jpg","publishedAt":"2017-10-05T19:30:00Z"},{"source":{"id":null,"name":"Lifehacker.com"},"author":"Alicia Adamczyk on Two Cents, shared by Alicia Adamczyk to Lifehacker","title":"Investing Is Risky and Unethical and You Should Do It Anyway","description":"Each Monday we’re tackling one of your pressing personal finance questions by asking a handf

In [57]:
!jupyter nbconvert NewsDataCleaning.ipynb --to slides --post serve